In [16]:
import logging

import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from pprint import pprint
import logging

import json
import pickle

In [34]:
pickle_path = os.path.join('..', '..', 'data', 'features_by_date')
features_by_date = pd.read_pickle(pickle_path)
features_by_date.columns

Index(['date', 'num_people_10_00', 'label_num_people_12_33',
       'label_num_people_11_30', 'label_num_menus_sold',
       'label_difference_12_normalized',
       'label_difference_10_and_sold_normalized', 'zurich_vacation',
       'exp_moving_avg_0.5', 'weekday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Spring/Autumn', 'Summer', 'Winter',
       'month_quarter_0', 'month_quarter_1', 'month_quarter_2',
       'month_quarter_3', 'zuehlke_day', 'Temperature', 'temp_deviation',
       'Rain Duration', 'Rain_half_discrete', 'Rain_binary',
       'average_of_past_weeks', 'deviation_1_day_ago', 'deviation_2_day_ago',
       'deviation_3_day_ago', 'deviation_4_day_ago'],
      dtype='object')

In [35]:
features_by_date[['exp_moving_avg_0.5','label_num_people_11_30']]

,exp_moving_avg_0.5,label_num_people_11_30
2022-03-14,134.500000,137
2022-03-15,144.750000,138
2022-03-16,146.000000,147
2022-03-17,135.500000,141
2022-03-18,64.250000,60
...,...,...
2022-11-21,202.745069,198
2022-11-22,233.167521,239
2022-11-23,241.862950,246
2022-11-24,219.885089,229


In [40]:
import math
math.isnan(np.nan)

True

In [14]:
def exponential_average_past_weeks(alpha=0.3):
    """alpha*new_val + (1-alpha)*old_val"""
    def func(day):
        num_people_11_30 = []
        for i in range(1, num_weeks + 1):
            considered_day = day - timedelta(days=i * 7)
            try:
                num_people_11_30.append(features_by_date.loc[considered_day, 'label_num_people_11_30'])
            except KeyError:
                print(f'{considered_day} gives KeyError')
        return np.mean(num_people_11_30)

    return func


In [26]:
alpha=0.3
for day in features_by_date.date:
    exp_mov_averages=pd.Series(index=features_by_date.date,dtype=float)
    try:
        avg_7_days_ago=exp_mov_averages[day-timedelta(days=7)]
    except KeyError: #start measurements, does not contain values of 7 days ago
        exp_mov_averages[day]=features_by_date.loc[day, 'label_num_people_11_30']
        try:
            exp_mov_averages[day]=alpha*features_by_date.loc[day, 'label_num_people_11_30']+(1-alpha)*avg_7_days_ago
        except KeyError: #no measurements due to public holiday,...
            exp_mov_averages[day]=avg_7_days_ago

exp_mov_averages

NameError: name 'avg_7_days_ago' is not defined

In [13]:
for day in features_by_date['date']:\
    print(day)


2022-03-09
2022-03-10
2022-03-11
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-23
2022-05-24
2022-05-25
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-11
2022-07-12
2022-07-13
2022-07-14
2022-07-15
2022-07-18
2022-07-19
2022-07-20
2022-07-21

In [4]:
day=date(year=2022,month=3,day=31)
num_people_11_30=[]
past_weeks=3
def average_of_past_weeks(num_weeks=3):
    def func(day):
        num_people_11_30=[]
        for i in range(1,num_weeks+1):
            considered_day=day-timedelta(days=i*7)
            try:
                num_people_11_30.append(features_by_date.loc[considered_day, 'label_num_people_11_30'])
            except KeyError:
                print(f'{considered_day} gives KeyError')
        return np.mean(num_people_11_30)
    return func

average_of_past_3_weeks=average_of_past_weeks(3)
average_of_past_3_weeks(day)
features_by_date['average_of_past_weeks']=features_by_date['date'].apply(average_of_past_3_weeks)

2022-03-02 gives KeyError
2022-02-23 gives KeyError
2022-02-16 gives KeyError
2022-03-03 gives KeyError
2022-02-24 gives KeyError
2022-02-17 gives KeyError
2022-03-04 gives KeyError
2022-02-25 gives KeyError
2022-02-18 gives KeyError
2022-03-07 gives KeyError
2022-02-28 gives KeyError
2022-02-21 gives KeyError
2022-03-08 gives KeyError
2022-03-01 gives KeyError
2022-02-22 gives KeyError
2022-03-02 gives KeyError
2022-02-23 gives KeyError
2022-03-03 gives KeyError
2022-02-24 gives KeyError
2022-03-04 gives KeyError
2022-02-25 gives KeyError
2022-03-07 gives KeyError
2022-02-28 gives KeyError
2022-03-08 gives KeyError
2022-03-01 gives KeyError
2022-03-02 gives KeyError
2022-03-03 gives KeyError
2022-03-04 gives KeyError
2022-03-07 gives KeyError
2022-03-08 gives KeyError
2022-04-15 gives KeyError
2022-04-18 gives KeyError
2022-04-15 gives KeyError
2022-04-18 gives KeyError
2022-04-15 gives KeyError
2022-04-18 gives KeyError
2022-05-26 gives KeyError
2022-05-27 gives KeyError
2022-05-26 g

C:\Users\mefl\AppData\Local\pypoetry\Cache\virtualenvs\food-saving-showcase-ocOVEVjy-py3.10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [7]:
day=date(year=2022,month=3,day=31)

def deviation_last_workday(day_delta=1):
    def func(day):
        row_num= features_by_date.index.get_loc(day)
        try:
            deviation_last_workday=(features_by_date.iloc[row_num-day_delta]['label_num_people_11_30']-features_by_date.iloc[row_num-day_delta]['average_of_past_weeks'])/features_by_date.iloc[row_num-day_delta]['average_of_past_weeks']
        except KeyError:
            deviation_last_workday=np.nan
        return deviation_last_workday
    return func

dev_1_day_ago=deviation_last_workday(1)
features_by_date['deviation_1_day_ago']=features_by_date['date'].apply(dev_1_day_ago)
dev_2_day_ago=deviation_last_workday(2)
features_by_date['deviation_2_day_ago']=features_by_date['date'].apply(dev_2_day_ago)
features_by_date[['date','deviation_1_day_ago','deviation_2_day_ago','label_num_people_11_30','average_of_past_weeks']]

,date,deviation_1_day_ago,deviation_2_day_ago,label_num_people_11_30,average_of_past_weeks
2022-03-09,2022-03-09,0.731092,0.042489,130,NaN
2022-03-10,2022-03-10,NaN,0.731092,128,NaN
2022-03-11,2022-03-11,NaN,NaN,50,NaN
2022-03-14,2022-03-14,NaN,NaN,137,NaN
2022-03-15,2022-03-15,NaN,NaN,138,NaN
...,...,...,...,...,...
2022-11-21,2022-11-21,-0.160350,-0.326452,198,212.333333
2022-11-22,2022-11-22,-0.067504,-0.160350,239,228.666667
2022-11-23,2022-11-23,0.045190,-0.067504,246,246.000000
2022-11-24,2022-11-24,0.000000,0.045190,229,219.666667


In [10]:
for i in features_by_date.columns:
    print(i)



date
num_people_10_00
label_num_people_12_33
label_num_people_11_30
label_num_menus_sold
label_difference_12_normalized
label_difference_10_and_sold_normalized
zurich_vacation
weekday
Monday
Tuesday
Wednesday
Thursday
Friday
Spring/Autumn
Summer
Winter
zuehlke_day
Temperature
temp_deviation
Rain Duration
Rain_half_discrete
Rain_binary
average_of_past_weeks
deviation_1_day_ago
deviation_2_day_ago


In [53]:
type(np.nan)

float

In [45]:
i=100
features_by_date.iloc[i-1]['label_num_people_11_30']-features_by_date.iloc[i-1]['average_of_past_3_weeks']

5.333333333333343

In [47]:
features_by_date.loc[day]

In [43]:
features_by_date.columns

Index(['date', 'num_people_10_00', 'label_num_people_12_33',
       'label_num_people_11_30', 'label_num_menus_sold',
       'label_difference_12_normalized',
       'label_difference_10_and_sold_normalized', 'zurich_vacation', 'weekday',
       'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Spring/Autumn',
       'Summer', 'Winter', 'zuehlke_day', 'Temperature', 'temp_deviation',
       'Rain Duration', 'Rain_half_discrete', 'Rain_binary', 'month_quarter_0',
       'month_quarter_1', 'month_quarter_2', 'month_quarter_3',
       'average_of_past_3_weeks'],
      dtype='object')

In [37]:
day=date(year=2022,month=3,day=17)

num_people_11_30=[]
num_weeks=3
for i in range(1,num_weeks+1):
    considered_day=day-timedelta(days=i*7)
    try:
        num_people_11_30.append(features_by_date.loc[considered_day, 'label_num_people_11_30'])
    except KeyError:
        print(f'{considered_day} gives KeyError')
np.mean(num_people_11_30)

2022-03-03 gives KeyError
2022-02-24 gives KeyError


128.0

In [ ]:
    for day in date_list_all:
        for i in relevant_num_days_ago:
            time_delta = timedelta(days=i)
            try:
                features_by_date.loc[day, (str(i) + '_days_ago_12_33')] = num_people_by_date.loc[
                    day - time_delta, 'num_people_at_12_33']
            except KeyError:
                features_by_date.loc[day, str(i) + '_days_ago_12_33'] = float('nan')
            try:
                features_by_date.loc[day, str(i) + '_days_ago_norm_dif'] = num_people_by_date.loc[
                    day - time_delta, 'normalized_difference_10_and_12']
            except KeyError:
                features_by_date.loc[day, str(i) + '_days_ago_norm_dif'] = float('nan')
